Step: Load in packages

In [12]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [13]:
import xgboost as xgb
import numpy as np
from skimpy import clean_columns
import pandas as pd
from pyprojroot import here
import os
import mlflow
import pandas as pd
import sklearn
from skopt import BayesSearchCV
from joblib import dump, load
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score, balanced_accuracy_score, f1_score
from sklearn.metrics import plot_precision_recall_curve
from sklearn.model_selection import train_test_split
import joblib
from analysis.fun_class_positive_predictions import fun_class_positive_predictions
from sklearn.model_selection import RepeatedKFold
from analysis.fun_classifer_xgboost_bayes import fun_classifer_xgboost_bayes
from analysis.fun_imbalanced_threshold_locator import fun_imbalanced_threshold_locator
from analysis.fun_class_accuracy_calcaulator import fun_class_accuracy_calcaulator
from analysis.fun_drop_high_correlation import fun_drop_high_correlation
import lightgbm as lgb
from analysis.fun_imbalanced_threshold import fun_imbalanced_threshold
from analysis.fun_class_predictions_id import fun_class_predictions_id

path_data = here("./data")
os.chdir(path_data)
data_trading_analysis = pd.read_parquet("data_trading_good_features.parquet")

Step: Add an id, because you are lazy want to use your pre-built function

In [14]:
data_trading_analysis_id = data_trading_analysis.reset_index()

Step: Remove highly correlated features

In [15]:
data_trading_analysis_low_corr = fun_drop_high_correlation(
    data=data_trading_analysis_id,
    outcome=["result"],
)

data_trading_analysis_low_corr.to_parquet("data_trading_analysis_low_corr.parquet")

Step: Split test and training

In [16]:
x = data_trading_analysis_low_corr.drop(columns=["result"])
y = data_trading_analysis_low_corr["result"]

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

Step: Get the imbalance measurement

In [18]:
from analysis.fun_scale_pos_weight import fun_scale_pos_weight

scale_pos_weight = fun_scale_pos_weight(y_train=y_train, outcome="result")
scale_pos_weight

0.2958984375

Step: Run ML 

In [19]:
results_xgb = fun_classifer_xgboost_bayes(
    scale_pos_weight=scale_pos_weight,
    x_train=x_train,
    y_train=y_train,
    n_splits=3,
    n_repeats=3,
    id_var="index",
)

path_outputs = here("./outputs")
os.chdir(path_outputs)
joblib.dump(results_xgb, "results_xgb.jlib")

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_enc

['results_xgb.jlib']

Step: Get predictions for all data and just training

In [20]:
data_prob_all = fun_class_positive_predictions(
    model=results_xgb,
    x_train_or_test=data_trading_analysis_low_corr,
    id_vars=["index", "result"],
)

data_prob_train = fun_class_positive_predictions(
    model=results_xgb,
    x_train_or_test=x_train,
    id_vars=["index"],
)

data_prob_test = fun_class_positive_predictions(
    model=results_xgb,
    x_train_or_test=x_test,
    id_vars=["index"],
)

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


Step: Get threshold for imbalanced data

In [21]:
threshold = fun_imbalanced_threshold(
    y_train=y_train, y_train_probability=data_prob_train
)
threshold

0.3648785

Step: Get classification for all data

In [22]:
data_trading_classification = fun_class_predictions_id(
    data_id=data_trading_analysis_low_corr["index"],
    outcome_probability=data_prob_all,
    threshold=threshold,
)

Step: Get predicted y for training

In [23]:
y_predicted = fun_imbalanced_threshold_locator(
    y_train=y_train,
    y_train_probability=data_prob_train,
    y_test_probability=data_prob_test,
)

Step: Get accuracy

In [24]:
results_accuracy = fun_class_accuracy_calcaulator(
    y_test=y_test, y_predicted=y_predicted
)
results_accuracy

[               metric_type  metric_score
 0  balanced_accuracy_score         0.112
 1            roc_auc_score         0.556
 2                 f1_score         0.815,
 {'balanced_accuracy': 0.11225328947368429,
  'roc_auc_score': 0.556126644736842,
  'f1_score': 0.8152380952380952}]